In [1]:
import os
import os.path
import networkx as nx
import json
import urllib
import traceback
from itertools import islice
from rdflib import Graph, URIRef, BNode, Namespace, Literal
from rdflib.namespace import RDF, OWL
from tqdm import tqdm
import json
import hashlib

In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
#import pheknowlator kg_utils 
import sys
sys.path.append('../')
from pkt_kg.utils import *

In [4]:
KG_PATH = '/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/'
MR_PATH = '/home/sanya/PheKnowLatorv2/machine_read/output_graphs/'
KG_NAME = 'PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle'
#MR_GRAPH_NAME = 'machineread_greentea_version1.gpickle'
MR_GRAPH_NAME_GT = 'machineread_greentea_version2.gpickle'
MR_GRAPH_NAME_KT = 'machineread_kratom_version1.gpickle'
NodeLabelsFilePL = 'PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_NodeLabels.txt'
NodeLabelsFileMR_gt = 'machineread_greentea_version2_NodeLabels.tsv'
NodeLabelsFileMR_kt = 'machineread_kratom_version1_NodeLabels.tsv'

In [5]:
with open(KG_PATH+'nodeLabels_20211014.pickle', 'rb') as filep:
    nodeLabels = pickle.load(filep)

In [6]:
pl_kg = nx.read_gpickle(KG_PATH+KG_NAME)

In [7]:
mr_kg = nx.read_gpickle(MR_PATH+MR_GRAPH_NAME_GT)
mr_kg2 = nx.read_gpickle(MR_PATH+MR_GRAPH_NAME_KT)

In [8]:
nx_graph = nx.compose_all([pl_kg, mr_kg, mr_kg2])

In [9]:
DIR_OUT = '/home/sanya/PheKnowLatorv2/output_files/'
obo = Namespace('http://purl.obolibrary.org/obo/')
napdi = Namespace('http://napdi.org/napdi_srs_imports:')

In [10]:
epicatechin = obo.CHEBI_90
catechin = obo.CHEBI_23053
egcg = obo.CHEBI_4806
greentea = napdi.camellia_sinensis_leaf
ecg = obo.CHEBI_70255
gallocatechin = obo.CHEBI_68330

#metformin
#diclofenac
midazolam = obo.CHEBI_6931
#dopamine
raloxifene = obo.CHEBI_8772
#amodiaquine
#bupripion
dextromethorphan = obo.CHEBI_4470
#digoxin
#atorvastatin
nadolol = obo.CHEBI_7444
#caffeine
#losartan
buspirone = obo.CHEBI_3223
#alprazolam

mitragynine = obo.CHEBI_6956
kratom = napdi.mitragyna_speciosa
hydroxy_mitragynine = napdi['7_hydroxy_mitragynine']

CYP3A4 = obo.PR_P08684
CYP2D6 = obo.PR_P10635
CYP2C19 = obo.PR_P33261
UGT1A1_protein = obo.PR_P22309
UGT1A8_protein = obo.PR_Q9HAW9
UGT1A10_protein = obo.PR_Q9HAW8

In [57]:
#get neighboring edges of the nodes
#get direct or indirect paths from NPs, NP constituents and drugs to enzymes
list(nx_graph.neighbors(greentea))
####catechin should also be in neighbors

[rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_90'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_68330'),
 rdflib.term.URIRef('http://napdi.org/napdi_srs_imports:camellia_sinensis_whole'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/PO_0025131'),
 rdflib.term.URIRef('http://napdi.org/napdi_srs_imports:epigallocatechin_gallate'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_42255'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_70255')]

In [67]:
x = list(nx_graph.neighbors(catechin))
for item in x:
    if 'napdi' in str(item):
        print(item)

http://napdi.org/napdi-srs-imports:camellia_sinensis_leaf


In [31]:
def get_neighbors(nx_graph, node):
    neighs = nx.neighbors(nx_graph, node)
    neighs_list = []
    for item in neighs:
        nodestr = nodeLabels[str(node)]
        neighbor = nodeLabels[str(item)]
        edge_keys = list(nx_graph.get_edge_data(node, item).keys())
        for edge in edge_keys:
            edgestr = nodeLabels[str(edge)]
            neighs_list.append((nodestr, edgestr, neighbor))
    return neighs_list

In [32]:
x = get_neighbors(nx_graph, greentea)

In [34]:
x

[('Camellia_sinensis_leaf', 'has component', '(-)-epicatechin'),
 ('Camellia_sinensis_leaf', 'has component', 'gallocatechin'),
 ('Camellia_sinensis_leaf', 'part of', 'Camellia_sinensis_whole'),
 ('Camellia_sinensis_leaf', 'subClassOf', 'plant anatomical entity'),
 ('Camellia_sinensis_leaf', 'has component', '(-)-epigallocatechin gallate'),
 ('Camellia_sinensis_leaf', 'has component', '(-)-epigallocatechin'),
 ('Camellia_sinensis_leaf', 'has component', '(-)-epicatechin-3-O-gallate')]

In [84]:
get_neighbors(nx_graph, kratom)

[('Mitragyna_speciosa', 'has component', 'Mitragynine'),
 ('Mitragyna_speciosa', 'part of', 'Mitragyna_speciosa_whole'),
 ('Mitragyna_speciosa', 'has component', '7-hydroxy-mitragynine'),
 ('Mitragyna_speciosa', 'subClassOf', 'plant anatomical entity')]

In [85]:
gtn = nx.neighbors(nx_graph, greentea)
greentea_list = []
for item in gtn:
    print(nodeLabels[str(item)])
    gtn_list = get_neighbors(nx_graph, item)
    greentea_list.extend(gtn_list)
len(greentea_list)

(-)-epicatechin
gallocatechin
Camellia_sinensis_whole
plant anatomical entity
(-)-epigallocatechin gallate
(-)-epigallocatechin
(-)-epicatechin-3-O-gallate


179

In [86]:
rels = []
greentea_data = {
'(-)-epicatechin': [],
'gallocatechin': [],
'(-)-epigallocatechin gallate': [],
'(-)-epigallocatechin': [],
'(-)-epicatechin-3-O-gallate': []
}
for item in greentea_list:
    if item[1] not in rels:
        rels.append(item[1])
    if item[0] in greentea_data:
        greentea_data[item[0]].append(item)

In [87]:
for key in greentea_data:
    print(key)
    print(len(greentea_data[key]))

(-)-epicatechin
68
gallocatechin
12
(-)-epigallocatechin gallate
1
(-)-epigallocatechin
7
(-)-epicatechin-3-O-gallate
87


In [101]:
epirel = []
for item in greentea_data['(-)-epicatechin']:
    if item[1] not in epirel:
        epirel.append(item[1])
print(epirel)

['is enantiomer of', 'subClassOf', 'has role', 'molecularly interacts with', 'directly negatively regulates activity of', 'IncreaseAmount', 'negatively regulates', 'Phosphorylation']


In [103]:
epid = {
    'is enantiomer of': [],
    'subClassOf': [], 
    'has role': [],
    'molecularly interacts with': [],
    'directly negatively regulates activity of': [],
    'IncreaseAmount': [],
    'negatively regulates': [],
    'Phosphorylation': []
}
for item in greentea_data['(-)-epicatechin']:
    if item[1] in epid:
        epid[item[1]].append(item[2])

In [112]:
epid['IncreaseAmount']

['mitogen-activated protein kinase 3',
 'voltage-gated potassium channel KCNH2',
 'taurochenodeoxycholate 6alpha-hydroxylase activity',
 'GRIA2_gene']

In [117]:
ktn = nx.neighbors(nx_graph, kratom)
kratom_list = []
for item in ktn:
    print(nodeLabels[str(item)])
    ktn_list = get_neighbors(nx_graph, item)
    kratom_list.extend(ktn_list)
len(kratom_list)

Mitragynine
Mitragyna_speciosa_whole
7-hydroxy-mitragynine
plant anatomical entity


73

In [118]:
rels = []
kratom_data = {
'Mitragynine': [],
'7-hydroxy-mitragynine': []
}
for item in kratom_list:
    if item[1] not in rels:
        rels.append(item[1])
    if item[0] in kratom_data:
        kratom_data[item[0]].append(item)

In [119]:
for key in kratom_data:
    print(key)
    print(len(kratom_data[key]))

Mitragynine
66
7-hydroxy-mitragynine
3


In [120]:
mitrel = []
for item in kratom_data['Mitragynine']:
    if item[1] not in mitrel:
        mitrel.append(item[1])
print(mitrel)

['subClassOf', 'interacts with', 'directly negatively regulates activity of', 'is substance that treats', 'IncreaseAmount', 'molecularly interacts with', 'negatively regulates']


In [122]:
mitd = {
    'interacts with': [],
    'subClassOf': [], 
    'is substance that treats': [],
    'molecularly interacts with': [],
    'directly negatively regulates activity of': [],
    'IncreaseAmount': [],
    'negatively regulates': []
}
for item in kratom_data['Mitragynine']:
    if item[1] in mitd:
        mitd[item[1]].append(item[2])

In [130]:
mitd['negatively regulates']

['potassium voltage-gated channel subfamily H member 2 (human)',
 'glycerol-3-phosphate phosphatase (human)',
 'protein',
 'prostaglandin G/H synthase 2 (human)',
 'ATP-dependent translocase ABCB1 (human)']

In [129]:
mitd['IncreaseAmount']

['cytochrome P450 1A2 (human)',
 'cytochrome P450 3A4 (human)',
 'potassium voltage-gated channel subfamily H member 2 (human)']

In [ ]:
#get neighbors of drugs

In [ ]:
#bidirectional shortest paths 

In [40]:
def get_path_labels(path):
    path_labels = []
    if len(path) < 1:
        print('Path length 1, skipping')
        return
    for edge in zip(path, path[1:]):
        data = nx_graph.get_edge_data(*edge)
        pred = list(data.keys())[0]
        node1_lab = str(edge[0])
        node2_lab = str(edge[1])
        if node1_lab in nodeLabels:
            node1_lab = nodeLabels[node1_lab]
        if node2_lab in nodeLabels:
            node2_lab = nodeLabels[node2_lab]
        pred_lab = nodeLabels[str(pred)]
        if list(data.values())[0]:
            if 'source_graph' in list(data.values())[0]:
                source_graph = 'machine_read'
            else:
                source_graph = ''
        else:
            source_graph = ''
        labels = [node1_lab, pred_lab, node2_lab, source_graph]
        path_labels.append(labels)
    return path_labels

In [131]:
pathx = nx.bidirectional_shortest_path(nx_graph, obo.CHEBI_6956, CYP3A4)
path_labels = get_path_labels(pathx)
for triples in path_labels:
    print(triples)

['Mitragynine', 'interacts with', 'cytochrome P450 3A4 (human)', '']
